In [18]:
from delphi_epidata import Epidata
import pandas as pd

In [19]:
# Data has to be acquired in batches since there is a length limit in the API. Acquiring in batches of 3 days

def retrieveData(month,start,end,df):
    if month < 10:
        month = f'0{month}'
    else:
        month = str(month)
    if start < 10:
        start = f'0{start}'
    else:
        start = str(start)
    if end < 10:
        end = f'0{end}'
    else:
        end = str(end)
        
    for i in range(int(start),int(end)+1,3):
        if i+1 < 10:
            if i == int(start):
                startdate = f'0{i}'
            else:
                startdate = f'0{i+1}'
        else:
            if i == int(start):
                startdate = i
            else:
                startdate = i+1
                
        enddate = i+3
        if i + 3 < 10:
            enddate = f'0{enddate}'
        
        if int(enddate) > int(end):
            enddate = int(end)
            
        startdate = str(startdate)
        enddate = str(enddate)
        
        if int(startdate) == int(enddate)+1:
            break
        
        res = Epidata.covidcast('doctor-visits', 'smoothed_cli', 'day', 'state', [Epidata.range(int(f'2020{month}{startdate}'), int(f'2020{month}{enddate}'))], geo_value='*')
        if res['result'] == 1:
            df = df.append(res['epidata'])
            print(f'retreival for {startdate} to {enddate} succeeded')
        else:
            print(f'Retreival for {startdate} to {enddate} failed')
    
    return df
        

In [20]:
df = retrieveData(4,8,30, pd.DataFrame())
df = retrieveData(5,1,28, df)

retreival for 08 to 11 succeeded
retreival for 12 to 14 succeeded
retreival for 15 to 17 succeeded
retreival for 18 to 20 succeeded
retreival for 21 to 23 succeeded
retreival for 24 to 26 succeeded
retreival for 27 to 29 succeeded
retreival for 30 to 30 succeeded
retreival for 01 to 04 succeeded
retreival for 05 to 07 succeeded
retreival for 08 to 10 succeeded
retreival for 11 to 13 succeeded
retreival for 14 to 16 succeeded
retreival for 17 to 19 succeeded
retreival for 20 to 22 succeeded
retreival for 23 to 25 succeeded
Retreival for 26 to 28 failed


In [21]:
counties = []
for county in pd.read_csv('../county_fips_codes.csv')['fips']:
    counties.append(county)
counties = list(set(counties))
cleanedDf = pd.DataFrame(dtype=str)

In [22]:
for fips in counties:
    info = pd.read_csv('../county_fips_codes.csv')
    state_code = info[info['fips'] == fips].values[0][2].lower()
    
    
    columns = df.loc[df['geo_value'] == state_code].values
    dates = []
    values = []
    for column in columns:
        date = column[1]
        date = list(str(date))
        date.insert(4, '-')
        date.insert(7, '-')
        date = ''.join(date)
        dates.append(date)
        values.append(column[3])
        
    values.insert(0,fips)
    dates.insert(0,'fips')
    
    temp = pd.DataFrame()
    temp = temp.append(pd.Series(values, index=dates), ignore_index=True)
    cleanedDf = cleanedDf.append(temp)

In [23]:
cleanedDf = cleanedDf.reindex(sorted(cleanedDf.columns), axis=1)
cleanedDf = cleanedDf.fillna(0)
firstCol = cleanedDf.pop('fips')
cleanedDf.insert(0, 'fips', firstCol)

In [24]:
cleanedDf.to_csv('doctorvisits.csv')